In [1]:
import numpy as np
import matplotlib.pyplot as plt
import opensimplex
import scipy.ndimage
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
opensimplex.seed(1234)


In [193]:
freq = 1
x, y = np.linspace(freq*-1, freq*1, 100), np.linspace(freq*-1, freq*1, 100)
z = opensimplex.noise2array(x, y)

array([[ 0.15528059,  0.16030812,  0.16561868, ...,  0.01685415,
         0.00526422, -0.0063771 ],
       [ 0.16101908,  0.16729515,  0.17379217, ...,  0.0458177 ,
         0.03432823,  0.02272583],
       [ 0.16685554,  0.17432985,  0.18195704, ...,  0.07457528,
         0.06322407,  0.05169512],
       ...,
       [ 0.01686392, -0.01214771, -0.04102528, ...,  0.03490883,
         0.01698108, -0.00075881],
       [ 0.00526487, -0.0238192 , -0.05277212, ...,  0.06488204,
         0.04755133,  0.03033239],
       [-0.0063771 , -0.03548001, -0.06444909, ...,  0.09545629,
         0.07879664,  0.062173  ]])

In [211]:
_b = np.linspace(-1, 1, 20)
_x, _y = np.meshgrid(_b, _b)
d = np.minimum(1, (_x**2+_x**2)/np.sqrt(2))


In [219]:
np.logspace(z.min(), z.max()+np.finfo(np.float32).eps, terrain_count+1, base=np.e)

array([0.55834679, 0.75004426, 1.00755732, 1.35348247, 1.81817426])

In [41]:
def population_map(freq=1, map_resolution=100, octaves=1, octave_factor=2, pop_threshold=0.5):
    z = np.zeros((map_resolution, map_resolution))
    of = 1
    for _ in range(octaves):
        x, y = np.linspace(freq*of*-1, freq*of*1, map_resolution), np.linspace(freq*of*-1, freq*of*1, map_resolution)
        z += (1/of)*opensimplex.noise2array(x, y)
        of *= octave_factor
        
    # Enable population islands
    znorm = (2*(z-np.min(z))/(np.max(z)-np.min(z)))-1
    _b = np.linspace(-1, 1, map_resolution)
    _x, _y = np.meshgrid(_b, _b)
    d = np.minimum(1, (_x**2+_y**2)/np.sqrt(2))
    z = (znorm + (1-d)) / 2
    z = (z-np.min(z))/(np.max(z)-np.min(z))
    z = np.digitize(z, [0, pop_threshold, 1])
    
    return z

def foliage_map(foliage_freq=1, map_resolution=100, foliage_octaves=1, foliage_octave_factor=2, foliage_levels=3):
    z = np.zeros((map_resolution, map_resolution))
    of = 1
    for _ in range(foliage_octaves):
        x, y = np.linspace(foliage_freq*of*-1, foliage_freq*of*1, map_resolution), np.linspace(foliage_freq*of*-1, foliage_freq*of*1, map_resolution)
        z += (1/of)*opensimplex.noise2array(x, y)
        of *= foliage_octave_factor
        
    z = (z-np.min(z))/(np.max(z)-np.min(z))
    z = np.digitize(z, np.linspace(z.min(), z.max(), foliage_levels+1))
    
    return z

In [42]:
def resolve_terrain(pop_map, fol_map):
    return fol_map

In [49]:
def generate(x0, y0, x1, y1, pop_freq, foliage_freq, pop_octaves, pop_octaves_factor, pop_threshold, foliage_levels, map_resolution=100):
    #-- Generate some data...
    pop_map = population_map(pop_freq, map_resolution, pop_octaves, pop_octaves_factor, pop_threshold)
    fol_map = foliage_map(foliage_freq, map_resolution, foliage_levels=foliage_levels)

    z = resolve_terrain(pop_map, fol_map)

    #-- Extract the line...
    # Make a line with "num" points...
    x1, y1 = int(x1*map_resolution), int(y1*map_resolution) # These are in _pixel_ coordinates!!
    x0, y0 = int(x0*map_resolution), int(y0*map_resolution)
    num = 10000
    x, y = np.linspace(x0, x1, num), np.linspace(y0, y1, num)
    # Extract the values along the line
    
    zi = z[y.astype(int), x.astype(int)]
    # zi = scipy.ndimage.map_coordinates(z, np.vstack((x,y)))

    # zi = np.digitize(zi, np.linspace(z.min(), z.max(), terrain_count+1), right=True)
    terrain_transitions = (np.abs(zi[1:] - zi[:-1]) > 0).astype(int)
    terrain_transitions = np.concatenate([np.array([0]), terrain_transitions])
    xtrans_coords = x[terrain_transitions == 1]
    ytrans_coords = y[terrain_transitions == 1]


    #-- Plot...
    fig, axes = plt.subplots(nrows=1, figsize=(10, 10))
    axes.imshow(z, origin='lower')
    axes.scatter(xtrans_coords, ytrans_coords, marker='x', color='r', s=500)
    axes.plot([x0, x1], [y0, y1], 'ro-')
    axes.axis('image')

    # axes[1].plot(zi)
    # axes[2].plot(terrain_transitions)

    plt.show()
interact(
    generate,
    map_resolution = widgets.IntSlider(min=100, max=1000, step=1, value=1000),
    foliage_levels = widgets.IntSlider(min=1, max=10, step=1, value=4),
    pop_octaves = widgets.IntSlider(min=1, max=10, step=1, value=4),
    pop_freq=widgets.FloatSlider(min=1, max=100, step=0.1, value=10),
    foliage_freq=widgets.FloatSlider(min=1, max=100, step=0.1, value=1),
    pop_octaves_factor=widgets.FloatSlider(min=1, max=10, step=0.1, value=1),
    pop_threshold=widgets.FloatSlider(min=0, max=1, step=0.1, value=0.5),
    
    x0=widgets.FloatSlider(min=0, max=1, step=0.1, value=0.1),
    y0=widgets.FloatSlider(min=0, max=1, step=0.1, value=0.1),
    x1=widgets.FloatSlider(min=0, max=1, step=0.1, value=0.9),
    y1=widgets.FloatSlider(min=0, max=1, step=0.1, value=0.9),
    
)

interactive(children=(FloatSlider(value=0.1, description='x0', max=1.0), FloatSlider(value=0.1, description='y…

<function __main__.generate(x0, y0, x1, y1, pop_freq, foliage_freq, pop_octaves, pop_octaves_factor, pop_threshold, foliage_levels, map_resolution=100)>